In [1]:
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.utils import check_array
import numpy as np

from numba import jit, vectorize, float64, int64
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.externals import joblib
from math import sqrt
from IPython import display
import dash
import dash_core_components as dcc
import dash_html_components as html
import warnings
import plotly.graph_objs as go

In [2]:
warnings.filterwarnings('ignore')

In [3]:
rf = joblib.load('./data/rf_trained.pkl')

In [4]:
# Data Loading
data = pd.read_csv('./data/TrainAndValid.csv')
test = pd.read_csv('./data/Test.csv')

# Date Features
tmp_date = data.saledate.apply(lambda x : datetime.strptime(x[:-5], '%m/%d/%Y'))
data['sale_mon'] = tmp_date.dt.month
data['sale_dayofweek'] = tmp_date.dt.dayofweek
data['sale_dayofyear'] = tmp_date.dt.dayofyear
data['sale_year'] = tmp_date.dt.year
data.drop(['saledate'],axis=1,inplace=True)

# Taking Subset of Columns
kept_columns = [
                'YearMade', 
                'sale_mon', 
                'sale_dayofweek',
                'sale_dayofyear',
                'sale_year',
                'fiModelDesc',
                'fiBaseModel',
                'fiProductClassDesc',
                'state',
                'SalePrice'
               ]
data = data[kept_columns]
data['age'] = data.sale_year - data.YearMade

# Encoding Class Description
data.loc[:,'classDesc_1'] = data.fiProductClassDesc.apply(lambda x : x.replace(',','').strip().split('-')[0])
data.loc[:,'classDesc_2'] = data.fiProductClassDesc.apply(lambda x : x.replace(',','').strip().split('-')[1])
data.drop('fiProductClassDesc',axis=1,inplace=True)

for col in ['fiModelDesc','fiBaseModel','state','classDesc_1','classDesc_2']:
    lb = LabelEncoder()
    data.loc[:,col] = lb.fit_transform(data.loc[:,col])

# Building a Random Forest Regressor
data.loc[data.YearMade < 1920, 'YearMade'] = np.median(data.YearMade)
data.loc[data.YearMade > 2012, 'YearMade'] = 2012

train, valid = train_test_split(data, test_size = .2)
y_train = train.SalePrice
y_valid = valid.SalePrice
train.drop('SalePrice',axis=1,inplace=True)
valid.drop('SalePrice',axis=1,inplace=True)


# Regression metrics

## MAE, MSE, MSPE, MAPE, MSLE, R2 

In [5]:
def mean_absolute_percentage_error(y_true, y_pred): 
    #y_true, y_pred = check_array(y_true, y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
def mean_squared_percentage_error(y_true, y_pred): 
    #y_true, y_pred = check_array(y_true, y_pred)
    return np.mean((y_true - y_pred)**2 / y_true) * 100

In [7]:
def get_regression_metrics(rf, train, y_train, valid, y_valid):
    #rf.fit(train, y_train)
    y_pred = rf.predict(valid)
    
    validation = {}
    validation["msle"] = mean_squared_log_error(y_valid, y_pred)
    validation["mae"] = mean_absolute_error(y_valid, y_pred)
    validation["mse"] = mean_squared_error(y_valid, y_pred)
    validation["mape"] = mean_absolute_percentage_error(y_valid, y_pred)
    validation["r2"] = r2_score(y_valid, y_pred)
    validation["rmsle"] = sqrt(mean_squared_log_error(y_valid, y_pred))
    validation["rmse"] = sqrt(mean_squared_error(y_valid, y_pred))
    validation["preds"] = y_pred
    #validation["rmspe"] = mean_squared_percentage_error(y_valid, y_pred)
    
    y_pred_train = rf.predict(train)
    training = {}
    training["msle"] = mean_squared_log_error(y_train, y_pred_train)
    training["mae"] = mean_absolute_error(y_train, y_pred_train)
    training["mse"] = mean_squared_error(y_train, y_pred_train)
    training["mape"] = mean_absolute_percentage_error(y_train, y_pred_train)
    training["r2"] = r2_score(y_train, y_pred_train)
    training["rmsle"] = sqrt(mean_squared_log_error(y_train, y_pred_train))
    training["rmse"] = sqrt(mean_squared_error(y_train, y_pred_train))
    training["preds"] = y_pred_train
    
    return validation, training

In [8]:
valid_metrics, train_metrics = get_regression_metrics(rf, train, y_train, valid, y_valid)

# show the Dash plot

In [10]:
def show_app(app,  # type: dash.Dash
             port=9999,
             width=700,
             height=350,
             offline=True,
             style=True,
             **dash_flask_kwargs):
    """
    Run the application inside a Jupyter notebook and show an iframe with it
    :param app:
    :param port:
    :param width:
    :param height:
    :param offline:
    :return:
    """
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url,
                                                                                  width=width,
                                                                                  height=height)
    display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False,  # needs to be false in Jupyter
                          port=port)
                          #**dash_flask_kwargs)

In [11]:
df_test = pd.DataFrame(valid_metrics).head(1).drop(["preds"], axis = 1).apply(lambda x:round(x, 4))

In [12]:
app_simple = dash.Dash()

In [18]:
app_simple.layout = html.Div(children=[
    html.H2(children='Alpha Release'),


    html.Table(
               [html.Tr([html.Th(col) for col in df_test.columns])] +
               [html.Tr( [html.Td(df_test.iloc[i][col]) for col in df_test.columns] ) for i in range(min(len(df_test), 10))],
               style={'border': '1px solid black',
                      'text-align': 'center',
                      'width':'100%'
                     }
              ),
    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                go.Scatter(x = valid_metrics["preds"][:20_000], y= y_valid[:20_000], 
                          mode='markers',
                          marker = {"opacity":0.5})
            ],
            'layout': {
                'title': 'Predicted vs. True Values',
                'xaxis':{'title':'Predicted'},
                'yaxis':{'title': 'True'}
            }
        }
    )
])

In [ ]:
show_app(app_simple,  # type: dash.Dash
             port=9999,
             width=1000,
             height=1000,
             offline=True,
             style=True)

 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2018 18:12:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2018 18:12:29] "GET /_dash-component-suites/dash_core_components/rc-slider@6.1.2.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2018 18:12:29] "GET /_dash-component-suites/dash_core_components/react-select@1.0.0-rc.3.min.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2018 18:12:29] "GET /_dash-component-suites/dash_core_components/react-virtualized@9.9.0.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2018 18:12:29] "GET /_dash-component-suites/dash_core_components/react-virtualized-select@3.1.0.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2018 18:12:29] "GET /_dash-component-suites/dash_core_components/react-dates@12.3.0.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2018 18:12:29] "GET /_dash-component-suites/dash_renderer/react@15.4.2.min.js?v=0.12.1 HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2018 18:12:29] "GET /_dash-componen